<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/08-NLP/How_To_Create_A_Powerful_TF_IDF_Keyword_Research_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fake-useragent

In [7]:
import pandas as pd
import numpy as np
import urllib
from fake_useragent import UserAgent
import requests
import re
from urllib.request import Request, urlopen
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import math
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = list(set(stopwords.words('english')))
from bs4 import BeautifulSoup
def get_text(url):
    try:
        req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req,timeout=5).read()
        soup = BeautifulSoup(webpage, "html.parser")
        texts = soup.findAll(text=True)
        res=u" ".join(t.strip() for t in texts if t.parent.name not in ['style', 'script', 'head', 'title', 'meta', '[document]'])
        return(res)
    except:
        return False


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
get_text('https://en.wikipedia.org/wiki/Machine_learning')[0:500]
#I will return the first 500 characters


'      CentralNotice    Machine learning   From Wikipedia, the free encyclopedia     Jump to navigation  Jump to search  For the journal, see Machine Learning (journal) .  "Statistical learning" redirects here. For statistical learning in linguistics, see statistical learning in language acquisition .  Scientific study of algorithms and statistical models that computer systems use to perform tasks without explicit instructions  Part of a series on Machine learning and data mining  Problems  Class'

Step 2: Get the URLs from competitors


In [26]:
def google_results(keyword, n_results):
    query = keyword
    query = urllib.parse.quote_plus(query) # Format into URL encoding
    number_result = n_results
    ua = UserAgent()
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
    response = requests.get(google_url, {"User-Agent": ua.random})
    soup = BeautifulSoup(response.text, "html.parser")
    result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
    results=[re.search('\/url\?q\=(.*)\&sa',str(i.find('a', href = True)['href'])) for i in result_div]
    links=[i.group(1) for i in results if i != None]
    return (links)

In [27]:
google_results('machine learning blog',10)

['https://towardsai.net/p/machine-learning/best-machine-learning-blogs-6730ea2df3bd',
 'https://machinelearningmastery.com/blog/',
 'https://blog.ml.cmu.edu/',
 'https://www.springboard.com/blog/machine-learning-blog/',
 'http://ai.googleblog.com/',
 'https://blog.feedspot.com/machine_learning_blogs/',
 'https://towardsdatascience.com/how-to-start-a-machine-learning-blog-in-a-month-7eaf84692df9',
 'https://deepmind.com/blog',
 'https://www.quora.com/What-are-the-best-regularly-updated-machine-learning-blogs-or-resources-available',
 'https://lionbridge.ai/articles/best-20-ai-and-machine-learning-blogs-to-follow-religiously/']

Step 3: Analyse the text and get the most important words.



In [28]:
def tf_idf_analysis(keyword_text):
  print(keyword_text)
  links=google_results(keyword_text,12)
  print(links)
  text=[]
  for i in links:
    t=get_text(i)
    if t:
      text.append(t)
    print(text)
            
  v = TfidfVectorizer(min_df=2,analyzer='word',ngram_range=(1,5),stop_words=stopWords)
  x = v.fit_transform(text)
  f = pd.DataFrame(x.toarray(), columns = v.get_feature_names())
  d=pd.concat([pd.DataFrame(f.mean(axis=0)),pd.DataFrame(f.max(axis=0))],axis=1)
    
    
  tf=pd.DataFrame((f>0).sum(axis=0))
  d=d.reset_index().merge(tf.reset_index(),on='index',how='left')
  d.columns=['word','average_tfidf','max_tfidf','frequency']
  #you can comment the following part if you want the number of URLs that the word occurs. The percentage makes sense
  #when we have a lot of URLs to check
  d['frequency']=round((d['frequency']/len(text))*100)
  return(d)


In [ ]:
x= tf_idf_analysis('machine learning blog')
#remove the numbers and sort by max tfidf and get the top20 words
x[x['word'].str.isalpha()].sort_values('max_tfidf',ascending=False).head(20)
